<a href="https://colab.research.google.com/github/yukihirop/.warp/blob/main/nlp_course_7_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip install accelerate==0.20.3
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


# Training a causal language model from scratch

これまでは、事前学習済みのモデルを使い、事前学習で得られた重みを再利用することで、新しいユースケース用にモデルを微調整してきました。第1章で見たように、これは一般的に転移学習と呼ばれ、ラベル付きデータがまばらなほとんどの実世界のユースケースにTransformerモデルを適用するための非常に成功した戦略です。この章では、異なるアプローチをとり、まったく新しいモデルをゼロから訓練します。これは、データがたくさんあり、利用可能なモデルに使用される事前学習データとは大きく異なる場合に取るべき良いアプローチです。しかし、言語モデルの事前学習には、既存のモデルを微調整するよりもかなり多くの計算リソースが必要になります。新しいモデルを訓練する意味がある例としては、音符からなるデータセット、DNAのような分子配列、プログラミング言語などがある。後者は、TabNineやGitHubのCopilotのような、OpenAIのCodexモデルを搭載したツールのおかげで、長いコード列を生成することができ、最近注目を集めている。このテキスト生成タスクは、GPT-2のような自己回帰的または因果的言語モデルで対処するのが最適です。


このセクションでは、コード生成モデルの縮小版を構築します。Pythonコードのサブセットを使用して、完全な関数やクラスの代わりに1行の補完に焦点を当てます。Pythonでデータを扱うとき、matplotlib、seaborn、pandas、scikit-learnライブラリで構成されるPythonデータサイエンススタックと頻繁に接触します。これらのフレームワークを使うとき、特定のコマンドを検索する必要があるのはよくあることなので、モデルを使ってこれらの呼び出しを完了できればいいのですが。

第6章では、Pythonソースコードを処理する効率的なトークナイザーを作成しましたが、やはり必要なのはモデルを事前学習するための大規模なデータセットです。ここでは、GitHubリポジトリから得られたPythonコードのコーパスにトークナイザーを適用します。そしてTrainer APIと🤗 Accelerateを使ってモデルを学習します。さっそく始めましょう！

これは、このセクションで示したコードを使用してトレーニングされ、ハブにアップロードされたモデルを示しています。ここにあります。テキスト生成でランダム化が行われているため、おそらく少し異なる結果が得られるでしょう。

## Gathering the data

PythonのコードはGitHubのようなコードリポジトリから豊富に入手可能であり、それを使ってPythonのリポジトリごとにスクレイピングしてデータセットを作成することができる。これは、Transformersの教科書で大規模なGPT-2モデルを事前訓練するために取られたアプローチです。codeparrotと呼ばれる約2000万のPythonファイルを含む約180GBのGitHubダンプを使用して、著者らはデータセットを構築し、その後Hugging Face Hubで共有した。

しかし、完全なコーパスのトレーニングは時間と計算量がかかるため、必要なのはPythonデータサイエンス・スタックに関係するデータセットのサブセットだけだ。そこで、まずcodeparrotデータセットから、このスタックに含まれるライブラリのいずれかを含むすべてのファイルをフィルタリングしてみよう。データセットのサイズが大きいので、ダウンロードは避けたい。代わりに、ストリーミング機能を使ってその場でフィルタリングする。その代わりに、ストリーミング機能を使って、その場でフィルタリングを行うことにする。先ほどのライブラリを使ったコードサンプルをフィルタリングするために、以下の関数を使う：

In [ ]:
def any_keyword_in_string(string, keywords):
  for keyword in keywords:
    if keyword in string:
      return True
  return False

2 つの例でテストしてみましょう。


In [ ]:
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]
example_1 = "import numpy as np"
example_2 = "import pandas as pd"

print(
    any_keyword_in_string(example_1, filters), any_keyword_in_string(example_2, filters)
)

False True


これを使って、データセットをストリーミングし、必要な要素をフィルタリングする関数を作ることができる：



In [ ]:
from collections import defaultdict
from tqdm import tqdm
from datasets import Dataset

def filter_streaming_dataset(dataset, filters):
  filtered_dict = defaultdict(list)
  total = 0
  for sample in tqdm(iter(dataset)):
    total += 1
    if any_keyword_in_string(sample["content"], filters):
      for k, v in sample.items():
        filtered_dict[k].append(v)
  print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering.")
  return Dataset.from_dict(filtered_dict)

そして、この関数をストリーミング・データセットに適用すればよい：

In [ ]:
# from datasets import load_dataset

# split = "train"
# filters = ["pandas", "sklearn", "matplotlib", "seaborn"]

# data = load_dataset(f"transformersbook/codeparrot-{split}", split=split, streaming=True)
# filtered_data = filter_streaming_dataset(data, filters)

この結果、データセットは6GBになり、600,000のPythonスクリプトで構成される！

全データセットのフィルタリングには、お使いのマシンと帯域幅にもよりますが、2～3時間かかります。このような長時間のプロセスをご自身で行いたくない場合は、フィルタリングされたデータセットをハブからダウンロードしていただけます：

In [ ]:
from datasets import load_dataset, DatasetDict

# cache_dir = '/content/drive/My Drive/nlp-76'

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(5000)),
        "valid": ds_valid.shuffle().select(range(50))
    }
)

raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 5000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 50
    })
})

言語モデルのプリトレーニングには時間がかかります。まず、上記の2つの部分行のコメントを解除して、データのサンプルでトレーニングループを実行し、トレーニングが正常に完了し、モデルが保存されていることを確認することをお勧めします。フォルダを作成し忘れたり、トレーニングループの最後にタイプミスがあったりして、最後のステップでトレーニングが失敗するほどイライラすることはありません！

データセットの例を見てみよう。各フィールドの最初の200文字だけを表示する：



In [ ]:
for key in raw_datasets["train"][0]:
  print(f"{key.upper()}: {raw_datasets['train'][0][key][:200]}")

REPO_NAME: hdmetor/scikit-learn
PATH: examples/datasets/plot_iris_dataset.py
COPIES: 283
SIZE: 1928
CONTENT: #!/usr/bin/python
# -*- coding: utf-8 -*-

"""
The Iris Dataset
This data sets consi
LICENSE: bsd-3-clause


contentフィールドには、モデルに学習させたいコードが含まれていることがわかる。データセットができたので、プレトレーニングに適した形式になるようにテキストを準備する必要がある。



## Preparing the dataset

最初のステップはデータをトークン化することで、トレーニングに使えるようにする。私たちの目的は主に短い関数呼び出しをオートコンプリートすることなので、コンテキストのサイズを比較的小さく保つことができる。これには、モデルをより速くトレーニングでき、必要なメモリが大幅に少なくて済むという利点がある。もしあなたのアプリケーションにとってより多くのコンテキストを持つことが重要であれば（例えば、関数定義のファイルに基づいて単体テストをモデルに書かせたい場合）、その数を増やすようにしてください。今のところ、コンテキスト・サイズを 128 トークンに固定しましょう。

ほとんどの文書には128個以上のトークンが含まれているので、入力を最大長に切り詰めるだけでは、データセットの大部分を削除してしまうことになる。その代わりに、return_overflowing_tokensオプションを使って、第6章でやったように、入力全体をトークン化していくつかのチャンクに分割する。また、return_lengthオプションを使用して、作成された各チャンクの長さを自動的に返します。最後のチャンクはコンテキストのサイズよりも小さくなることがよくあるので、パディングの問題を避けるためにこれらの部分を取り除きます。

最初の2つの例で、これがどのように機能するかを見てみよう：



In [ ]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {outputs['length']}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Input IDs length: 31
Input chunk lengths: [128, 128, 128, 128, 123, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 99]
Chunk mapping: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


この 2 つの例から、合計で 34 のセグメントが得られたことがわかります。チャンクの長さを見ると、両方のドキュメントの末尾にあるチャンクのトークンが 128 未満であることがわかります (それぞれ 117 と 41)。これらはチャンク全体のほんの一部なので、捨てても大丈夫です。overflow_to_sample_mappingフィールドを使えば、どのチャンクがどの入力サンプルに属するかを再構築することもできます。

この操作では、🤗 DatasetsのDataset.map()関数の便利な機能である、1対1のマップを必要としないことを利用しています。3節で見たように、入力バッチよりも要素の多いバッチや少ないバッチを作成することができます。これは、データ拡張やデータフィルタリングのような、要素数を変更する操作を行うときに便利です。今回のケースでは、各要素を指定されたコンテキストサイズのチャンクにトークン化する際に、各ドキュメントから多くのサンプルを作成します。既存のカラムはサイズが競合するので、必ず削除する必要があります。もしそれらを残したければ、Dataset.map()呼び出しの中でそれらを適切に繰り返して返すことができます：

In [ ]:
def tokenize(element):
  outputs = tokenizer(
      element['content'],
      truncation=True,
      max_length=context_length,
      return_overflowing_tokens=True,
      return_length=True
  )
  input_batch = []
  for length, input_ids in zip(outputs['length'], outputs['input_ids']):
    if length == context_length:
      input_batch.append(input_ids)
  return {"input_ids": input_batch}

tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets['train'].column_names
)
tokenized_datasets

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 133179
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 1404
    })
})

現在、各トークンが 128 個の 1670 万の例があり、これは合計約 21 億トークンに相当します。参考までに、OpenAIのGPT-3とCodexモデルはそれぞれ3,000億と1,000億のトークンで学習されており、CodexモデルはGPT-3のチェックポイントから初期化されています。このセクションでの我々のゴールは、長くて首尾一貫したテキストを生成できるこれらのモデルと競合することではなく、データサイエンティストのための迅速なオートコンプリート機能を提供する縮小版を作成することである。

データセットの準備ができたので、モデルをセットアップしよう！

✏️ 試してみよう！ここでは小さなコンテキストウィンドウを使っているので、コンテキストサイズより小さいチャンクをすべて取り除くことは大きな問題ではありませんでした。コンテキストサイズを大きくすると（あるいは短い文書のコーパスを使うと）、捨てられるチャンクの割合も大きくなる。より効率的にデータを準備する方法は、トークン化されたサンプルをeos_token_idトークンを挟んで一括で連結し、連結されたシーケンスに対してチャンキングを実行することです。練習として、tokenize() 関数を修正して、この方法を利用してください。トークンIDの完全なシーケンスを取得するために、truncation=Falseを設定し、トークナイザから他の引数を削除することに注意してください。

## Initializing a new model

最初のステップはGPT-2モデルを初期化することです。私たちのモデルには、小さなGPT-2モデルと同じ設定を使用します。そのため、事前学習された設定をロードし、トークナイザーのサイズがモデルの語彙サイズと一致することを確認し、bosとeos（シーケンスの開始と終了）のトークンIDを渡します：



In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

この設定で、新しいモデルをロードすることができます。これは、実際に自分自身でモデルを初期化するため、from_pretrained()関数を使用しない最初の例であることに注意してください：



In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


我々のモデルには124Mのパラメータがあり、チューニングする必要がある。トレーニングを始める前に、バッチを作成するデータ・コレーターをセットアップする必要がある。言語モデリング専用に設計されたDataCollatorForLanguageModelingコレーターを使うことができる（名前が微妙に示唆しているように）。因果言語モデリングでは、入力もラベルの役割を果たしますが（要素が1つずれるだけです）、このデータ照合器は学習中にその場でラベルを作成するので、input_idを複製する必要はありません。

DataCollatorForLanguageModelingは、マスク言語モデリング（MLM）と因果言語モデリング（CLM）の両方をサポートしていることに注意してください。デフォルトではMLM用にデータを準備しますが、引数mlm=Falseを設定することでCLMに切り替えることができます：


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

例を見てみよう：



In [ ]:
# def tokenize_function(examples):
#   return tokenizer(examples["input_ids"], padding="max_length", truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
  print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


例題が積み重ねられ、すべてのテンソルが同じ形をしていることがわかる。

⚠️ 入力とラベルの位置を合わせるためのシフトはモデル内部で行われるので、データ・コレーターはラベルを作成するために入力をコピーするだけである。

これで、実際にモデルをトレーニングするための準備がすべて整いました！トレーニングを始める前に、Hugging Faceにログインしよう。ノートブックで作業しているなら、次のユーティリティ関数でログインできます：

あとは学習引数を設定し、Trainerを起動するだけです。いくつかのウォームアップと256（per_device_train_batch_size * gradient_accumulation_steps）の有効バッチサイズを持つコサイン学習率スケジュールを使用します。勾配累積は、1つのバッチがメモリに収まらない場合に使用され、いくつかのフォワード/バックワードパスを通して勾配を増分的に積み上げます。これは🤗 Accelerateでトレーニングループを作成するときに実際に見ることができます。

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True
)

if torch.cuda.is_available():
  model.cuda()

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

あとはトレーナーを起動して、トレーニングが終わるのを待つだけだ。トレーニングセットの全部で実行するかサブセットで実行するかにもよるが、それぞれ20時間か2時間かかる！



In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=520, training_loss=5.394811542217548, metrics={'train_runtime': 1663.9053, 'train_samples_per_second': 80.04, 'train_steps_per_second': 0.313, 'total_flos': 8695798824960000.0, 'train_loss': 5.394811542217548, 'epoch': 1.0})

トレーニングが完了したら、モデルとトークナイザをハブにプッシュすることができる：

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1705326904.83069263d373.18843.0:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/yukihirop/codeparrot-ds/commit/4dc8d21f71a99af36f56bf70491cab17bc278b40', commit_message='End of training', commit_description='', oid='4dc8d21f71a99af36f56bf70491cab17bc278b40', pr_url=None, pr_revision=None, pr_num=None)

✏️ 試してみてください！生テキストからGPT-2のトレーニングまで、TrainingArgumentsに加えて約30行のコードで済みました。自分のデータセットで試してみて、良い結果が得られるかどうか確かめてみてください！

💡 複数のGPUを搭載したマシンにアクセスできる場合は、そこでコードを実行してみてください。Trainerは自動的に複数のマシンを管理し、トレーニングを飛躍的にスピードアップします。

## Code generation with a pipeline

さて、いよいよ正念場だ。学習させたモデルが実際にどの程度機能するのか見てみよう！ログを見ると、損失が着実に減少していることがわかるが、モデルをテストするために、いくつかのプロンプトでどの程度機能するか見てみよう。そのために、モデルをテキスト生成パイプラインでラップし、利用可能であれば高速生成のためにGPUに載せます：

In [ ]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="huggingface-course/codeparrot-ds", device=device
)

config.json:   0%|          | 0.00/938 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

散布図を作るという簡単な作業から始めよう：



In [ ]:
txt = """\
# create some data
x = np.random().randn(100)
y = np.random().randn(100)

# create scatter plot with x, y
"""

print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# create some data
x = np.random().randn(100)
y = np.random().randn(100)

# create scatter plot with x, y
plt.scatter(x, y, color='b')



結果は正しく見えます。pandasの操作でもうまくいくだろうか？2つの配列からDataFrameを作成できるか見てみましょう：



In [ ]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
df = pd.DataFrame(x, columns=[0, 1])


ナイス、これが正解だ。ただし、x列が再び挿入される。生成されるトークンの数は限られているので、次のforループは切り捨てられます。もう少し複雑なことをして、モデルにgroupbyオペレーションを使わせてみましょう：



In [ ]:
txt = """\
# dataframe with profession, income and name
df = pd.DataFrame({'profesion': x, 'income': y, 'name': z})

# calculate the mean income per profession
"""

print(pipe(txt, num_return_sequences=2)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# dataframe with profession, income and name
df = pd.DataFrame({'profesion': x, 'income': y, 'name': z})

# calculate the mean income per profession
profession[x


悪くない。これが正しいやり方だ。最後に、これをscikit-learnにも使って、Random Forestモデルをセットアップできるか見てみよう：



In [ ]:
txt = """\
# import random forest regressor from skit-learn
from sklearn.ensemble import RandomForestRegressor

# fit random forest model with 300 estimators on X, y:
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# import random forest regressor from skit-learn
from sklearn.ensemble import RandomForestRegressor

# fit random forest model with 300 estimators on X, y:
try:
    import sklearn.ensemble
    import sklearn.preprocessing
    import sklearn


これらのいくつかの例を見ると、モデルはPythonデータサイエンス・スタックの構文をある程度学習しているようだ（もちろん、モデルを実世界にデプロイする前に、もっと徹底的に評価する必要があるだろう）。しかし、あるユースケースに必要なパフォーマンスを達成するために、モデル学習をよりカスタマイズする必要がある場合もあります。例えば、バッチサイズを動的に更新したい場合や、悪い例をその場でスキップする条件付き学習ループを持ちたい場合はどうすればいいでしょうか？Trainerをサブクラス化し、必要な変更を加えるという方法もありますが、トレーニングループをゼロから書く方が簡単な場合もあります。そこで🤗 Accelerateの登場です。



## Training with Accelerate

Trainerを使ってモデルをトレーニングする方法について見てきました。しかし、トレーニングループを完全にコントロールしたい場合や、エキゾチックな変更を加えたい場合もあります。このような場合、🤗 Accelerateは最適な選択です。このセクションでは、モデルをトレーニングするためにAccelerateを使用する手順を説明します。さらに面白くするために、トレーニングループに一工夫加えます。

我々は主にデータサイエンスライブラリのための賢明なオートコンプリートに興味があるので、これらのライブラリをより多く使用するトレーニングサンプルをより重視することは理にかなっています。これらの例は、matplotlib.pyplot、pandas、sklearnのインポート名として最も頻度の高いplt、pd、sk、fit、predictや、後者のfit/predictパターンなどのキーワードを使うことで簡単に特定できます。これらがそれぞれ1つのトークンとして表現されていれば、入力シーケンスに出現するかどうかを簡単にチェックすることができます。トークンは空白の接頭辞を持つことができるので、トークナイザーの語彙にそれらのバージョンがあるかどうかもチェックします。動作確認のため、複数のトークンに分割されるはずのテストトークンを1つ追加します：

In [ ]:
keytoken_ids = []
for keyword in [
    "plt",
    "pd",
    "sk",
    "fit",
    "predict",
    " plt",
    " pd",
    " sk",
    " fit",
    " predict",
    "testtest"
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
      keytoken_ids.append(ids[0])
    else:
      print(f"keyword has not single token: {keyword}")

keyword has not single token: testtest


うまくいったようだ！これで、入力シーケンス、ロジット、そして先ほど選択したキートークンを入力とするカスタム損失関数を書くことができる。次のトークンは現在のトークンのラベルなので、入力シーケンスが右に1つシフトしたものがラベルになります。入力シーケンスの2番目のトークンからラベルを開始することで、これを達成することができる。そして最後のロジットをカットする。入力シーケンスの全トークンに続くトークンのラベルがないからだ。これでサンプルごとの損失を計算し、各サンプルですべてのキーワードの出現をカウントすることができる。最後に、出現回数を重みとして全サンプルの加重平均を計算する。キーワードのないサンプルをすべて捨てたくないので、重みに1を加える：

このコードの部分は、言語モデルのトレーニングにおいて、予測タスクを正しく設定するために使用される処理です。具体的には、トレーニングデータのラベル（正解）と予測（モデルの出力）を適切にシフト（ずらす）しています。このシフトは、モデルが各トークンの次に来るトークンを予測するようにするために行われます。それぞれのステップを詳しく説明します：

1. **ラベルのシフト**：
   `shift_labels = inputs[..., 1:].contiguous()`
   ここでの`inputs`はモデルに入力されるトークンのシーケンスを表しています。`inputs[..., 1:]`は、シーケンスの各トークンを1つずつシフトしています。つまり、各入力トークンに対して、その「次の」トークンが正解ラベルとして使われるようにしています。たとえば、元のシーケンスが`[A, B, C, D]`だった場合、シフト後のラベルは`[B, C, D]`になります。このシフトにより、モデルは各トークン`A`, `B`, `C`が与えられた時、それぞれ`B`, `C`, `D`を予測するようになります。

2. **予測（ロジット）のシフト**：
   `shift_logits = logits[..., :-1, :].contiguous()`
   ここでの`logits`はモデルが出力する予測値（ロジット）です。この予測値は、通常、入力シーケンスの各トークンに対する次のトークンの予測スコアを含んでいます。`logits[..., :-1, :]`により、予測値のシーケンスを最後のトークンを除いてシフトします。これにより、各入力トークンに対して、その「次の」トークンの予測スコアが対応するようになります。

3. **`contiguous`メソッド**：
   `contiguous()`メソッドは、メモリ上で連続したテンソルを確保するために使われます。PyTorchでは、テンソルの操作（例えばスライスやシフト）を行うと、結果のテンソルがメモリ上で非連続（分断された）状態になることがあります。`contiguous()`メソッドを使うことで、テンソルがメモリ上で連続的に配置されることが保証され、後続の操作（例えば`.view()`）が効率的に行えるようになります。

このシフト処理により、言語モデルが各入力トークンに対して適切な「次のトークン」を予測するタスクが設定されます。これは、自然言語処理における一般的なアプローチであり、特にトランスフォーマーベースのモデル（GPT-2など）でよく使用されます。

In [ ]:
from torch.nn import CrossEntropyLoss
import torch

# モデルの予測（shift_logits）と実際のラベル（shift_labels）

def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
  # Shift so that tokens < n predict n
  shift_labels = inputs[..., 1:].contiguous()
  shift_logits = logits[..., :-1, :].contiguous()
  # calculate per-token loss
  loss_fct = CrossEntropyLoss(reduce=False)
  loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
  # resize and average loss per sample
  loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
  # calculate and scale weighting
  weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
      axis=[0, 2]
  )
  weights = alpha * (1.0 + weights)
  # calculate weighted average
  weighted_loss = (loss_per_sample * weights).mean()
  return weighted_loss

この素晴らしい新しい損失関数を使ってトレーニングを始める前に、いくつかのことを準備する必要がある：

データをバッチでロードするためのデータ・ローダーが必要である。
重み減衰パラメータを設定する必要がある。
時々評価したいので、評価コードを関数でラップするのは理にかなっている。
データ・ローダーから始めよう。データセットのフォーマットを "torch "に設定するだけで、適切なバッチサイズでPyTorch DataLoaderに渡すことができます：

In [ ]:
from torch.utils.data.dataloader import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader= DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["valid"], batch_size=32)

次に、オプティマイザがどのパラメータにウェイト減衰がかかるかを認識できるように、パラメータをグループ化します。通常、バイアスとレイヤーノームのウェイト項はすべてこの対象外です：



この関数を使用する目的は、モデルのトレーニング中に特定のパラメータ（例えばバイアスやLayerNormの重み）に重み減衰を適用しないことです。これは、これらの特定のパラメータが重み減衰によって過度に制約されることを防ぐために行われます。重み減衰は、過学習を防ぐためにモデルの重みを小さく保つ一般的な手法ですが、全てのパラメータに一律に適用するのではなく、選択的に適用することが重要です。

In [ ]:
weight_decay = 0.1

def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
  params_with_wd, params_without_wd = [], []
  for n, p in model.named_parameters():
    if any(nd in n for nd in no_decay):
      params_without_wd.append(p)
    else:
      params_with_wd.append(p)
  return [
      {"params": params_with_wd, "weight_decay": weight_decay},
      {"params": params_without_wd, "weight_decay": 0.0}
  ]



トレーニング中に検証セットで定期的にモデルを評価したいので、そのための関数も書きましょう。この関数は、評価データローダを実行し、プロセス全体の損失を収集します：



この関数は、モデルの性能を評価し、特に言語モデルの性能をパープレキシティという指標で測定するために使用されます。パープレキシティは、モデルがテキストの次の単語をどれだけ正確に予測できるかを示します。低いパープレキシティは、モデルがデータをより正確にモデリングしていることを意味します。








In [ ]:
def evaluate():
  model.eval()
  losses = []
  for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
      outputs = model(batch["input_ids"], labels=batch["input_ids"])

    # 損失を1次のテンソルに変換
    losses.append(accelerator.gather(outputs.loss).unsqueeze(0))

  # 修正: 1次元テンソルのリストを連結
  loss = torch.mean(torch.cat(losses))
  try:
    perplexity = torch.exp(loss)
  except OverflowError:
    perplexity = float("inf")
  return loss.item(), perplexity.item()


evaluate()関数を使えば、損失と当惑度を一定間隔で報告することができる。次に、モデルを再定義して、もう一度ゼロから訓練するようにします：



In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


我々のモデルには124Mのパラメータがあり、チューニングする必要がある。トレーニングを始める前に、バッチを作成するデータ・コレーターをセットアップする必要がある。言語モデリング専用に設計されたDataCollatorForLanguageModelingコレーターを使うことができる（名前が微妙に示唆しているように）。因果言語モデリングでは、入力もラベルの役割を果たしますが（要素が1つずれるだけです）、このデータ照合器は学習中にその場でラベルを作成するので、input_idを複製する必要はありません。

DataCollatorForLanguageModelingは、マスク言語モデリング（MLM）と因果言語モデリング（CLM）の両方をサポートしていることに注意してください。デフォルトではMLM用にデータを準備しますが、引数mlm=Falseを設定することでCLMに切り替えることができます：

次に、前の関数を使用して重み減衰のパラメーターを分割し、オプティマイザーを定義できます。


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(model), lr=5e-4)

それでは、モデル、オプティマイザー、データ・ローダーを準備し、トレーニングを開始しましょう：



In [ ]:
from accelerate import Accelerator

help(Accelerator)

accelerator = Accelerator()

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Help on class Accelerator in module accelerate.accelerator:

class Accelerator(builtins.object)
 |  Accelerator(device_placement: 'bool' = True, split_batches: 'bool' = False, mixed_precision: 'PrecisionType | str | None' = None, gradient_accumulation_steps: 'int' = 1, cpu: 'bool' = False, deepspeed_plugin: 'DeepSpeedPlugin | None' = None, fsdp_plugin: 'FullyShardedDataParallelPlugin | None' = None, megatron_lm_plugin: 'MegatronLMPlugin | None' = None, rng_types: 'list[str | RNGType] | None' = None, log_with: 'str | LoggerType | GeneralTracker | list[str | LoggerType | GeneralTracker] | None' = None, project_dir: 'str | os.PathLike | None' = None, project_config: 'ProjectConfiguration | None' = None, gradient_accumulation_plugin: 'GradientAccumulationPlugin | None' = None, dispatch_batches: 'bool | None' = None, even_batches: 'bool' = True, step_scheduler_with_optimizer: 'bool' = True, kwargs_handlers: 'list[KwargsHandler] | None' = None, dynamo_backend: 'DynamoBackend | str | None' = 

train_dataloaderをaccelerator.prepare()に送ったので、その長さを使ってトレーニングステップ数を計算します。このメソッドはその長さを変更するので、常にdataloaderを準備した後にこれを行うべきであることを覚えておいてください。学習率から0までの古典的な線形スケジュールを使用します：



In [ ]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name = "linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps
)

最後に、モデルをハブにプッシュするために、作業フォルダにRepositoryオブジェクトを作成する必要があります。まだログインしていなければ、まずHugging Face Hubにログインしてください。モデルに与えたいモデルIDからリポジトリ名を決定します（リポジトリ名は自由に置き換えてください：

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "codeparrot-ds-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'yukihirop/codeparrot-ds-accelerate'

そして、そのリポジトリをローカルフォルダーにクローンします。すでに存在するのであれば、このローカルフォルダは作業中のリポジトリの既存のクローンでなければなりません：

repo.push_to_hub()メソッドを呼び出すことで、output_dirに保存したものをアップロードできるようになった。これにより、各エポックの終わりに中間モデルをアップロードすることができます。

学習する前に、評価関数が正しく動くかどうか簡単なテストをしてみましょう：

In [ ]:
output_dir = "codeparrot-ds-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/yukihirop/codeparrot-ds-accelerate into local empty directory.


In [ ]:
evaluate()

(3.7600059509277344, 42.94868087768555)

損失と当惑度は非常に高い値だが、まだモデルをトレーニングしていないのだから当然だろう。これで、トレーニングスクリプトの核となる部分、トレーニングループを書く準備がすべて整いました。トレーニングループでは、dataloaderを繰り返し処理し、バッチをモデルに渡します。ロジットがあれば、カスタム損失関数を評価することができます。より多くのステップを集約する際に損失が大きくならないように、勾配累積ステップ数で損失をスケーリングします。最適化する前に、収束を良くするために勾配を切り取ります。最後に、数ステップごとに、新しいevaluate()関数を使って評価セットでモデルを評価します：

このコードは、機械学習のトレーニングプロセスを実装しています。主に、特定のモデルに対してトレーニングデータを使用して学習し、定期的に評価を行い、結果を保存する流れです。コードの主要な部分を順に説明します：

1. **インポートと変数の設定**:
   - `from tqdm.notebook import tqdm`: `tqdm`は進捗バーを表示するライブラリです。`notebook`バージョンはJupyterノートブックで使われることが多いです。
   - `gradient_accumulation_steps`: 勾配蓄積のステップ数です。これは、勾配を更新する前に何回のバックプロパゲーションを蓄積するかを決定します。
   - `eval_steps`: 評価を実行するステップの間隔です。

2. **モデルのトレーニング**:
   - `model.train()`: モデルをトレーニングモードに設定します。
   - `for epoch in range(num_train_epochs)`: 指定されたエポック数だけトレーニングを繰り返します。
   - `for step, batch in tqdm(...)`: トレーニングデータローダーを通じて、各バッチを進行状況バーと共に反復処理します。

3. **損失の計算とバックプロパゲーション**:
   - `logits = model(batch["input_ids"]).logits`: モデルを使用してバッチの予測を行い、ロジット（クラスごとのスコア）を取得します。
   - `loss = keytoken_weighted_loss(...)`: 特定のキートークンに基づいて重み付けされた損失関数を計算します。
   - `accelerator.backward(loss)`: 損失に基づいてモデルの勾配を計算します。`accelerator`は、分散トレーニングを簡素化するためのユーティリティです。

4. **勾配蓄積とパラメータの更新**:
   - 勾配を蓄積し、`gradient_accumulation_steps`ごとにモデルのパラメータを更新します。
   - `optimizer.step()`と`lr_scheduler.step()`でオプティマイザと学習率スケジューラを更新します。

5. **定期的な評価とモデルの保存**:
   -

   - `if (step % (eval_steps * gradient_accumulation_steps)) == 0:`: 特定のステップ間隔ごとに評価を実行します。
   - `eval_loss, perplexity = evaluate()`: 評価関数を使用して損失とパープレキシティ（モデルの予測がどれだけ良いかを示す指標）を計算します。
   - `accelerator.print(...)`: 計算された評価指標を表示します。
   - `unwrapped_model = accelerator.unwrap_model(model)`: モデルをアンラップして保存用に準備します。
   - `unwrapped_model.save_pretrained(...)`: トレーニングされたモデルを保存します。
   - `tokenizer.save_pretrained(output_dir)`: トークナイザー（テキストをトークンに変換するためのツール）も保存します。
   - `repo.push_to_hub(...)`: トレーニング状態をHugging Face Hubにプッシュします。これは、モデルのバージョン管理と共有を容易にするためです。

6. **その他の詳細**:
   - `accelerator`: これは、Hugging Face Accelerate ライブラリからのもので、GPUやTPUなどのハードウェアアクセラレーションを簡単に利用するためのツールです。
   - `if step % 100 == 0:`: 毎100ステップごとに途中結果を表示します。

このコードの全体的な目的は、機械学習モデルの効率的なトレーニングと評価、そしてその結果の保存と共有です。また、このコードは、特にHugging Faceのツールを利用しており、その環境で最適化されています。

この行のコード `loss = loss / gradient_accumulation_steps` は、勾配蓄積(Gradient Accumulation)のプロセスの一部です。勾配蓄積は、特にGPUメモリが限られている場合に有効なテクニックで、大きなバッチサイズを模倣するために使用されます。具体的には、以下の手順で行われます：

1. **小さなバッチでの順伝播と損失計算**：まず、通常通りに小さなバッチをモデルに入力し、予測を行い、損失を計算します。

2. **損失の平均化**：この行では、計算された損失を `gradient_accumulation_steps` で割っています。これは、複数のバッチにわたって蓄積される勾配の量を平均化するために行われます。例えば、`gradient_accumulation_steps` が 8 の場合、各バッチの損失は 1/8 にスケーリングされます。

3. **勾配の蓄積**：スケーリングされた損失に基づいてバックプロパゲーションが行われ、勾配が計算されますが、この時点ではまだモデルの重みは更新されません。

4. **重みの更新**：`gradient_accumulation_steps` 回のバックプロパゲーションが完了した後、蓄積された勾配を使用して一度にモデルの重みを更新します。

このプロセスにより、大きなバッチサイズをシミュレートすることができ、GPUメモリの制約により直接大きなバッチサイズを使用できない場合でも、類似の効果を得ることができます。

In [ ]:
from tqdm.notebook import tqdm

# 勾配蓄積のステップ数です。これは、勾配を更新する前に何回のバックプロぱゲーションを蓄積するかを決定します。
gradient_accumulation_steps = 8
# 評価を実行するステップの感覚です。
eval_steps = 5_000

# モデルをトレーニングモードにする
model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
  for step, batch in tqdm(
      enumerate(train_dataloader, start=1), total=num_training_steps
  ):
    # 損失計算とバックプロpageーション
    # モデルを使用してバッチの予測を行い、ロジット（クラスごとのスコア)を取得する。
    logits = model(batch["input_ids"]).logits
    # 特定のキートクンに基づいて重みづけされた損失関数を計算します。
    loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
    if step % 100 == 0:
      accelerator.print(
          {
              "steps": completed_steps,
              "loss/train": loss.item() * gradient_accumulation_steps
          }
      )
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0:
      accelerator.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      completed_steps += 1
    if (step % (eval_steps * gradient_accumulation_steps)) == 0:
      eval_loss, perplexity = evaluate()
      accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
      model.train()
      accelerator.wait_for_everyone()
      unwrapped_model = accelerator.unwrap_model(model)
      unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
      if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress step {step}", blocking=False
        )


  0%|          | 0/4162 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 11>:17                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518 in _wrapped_call_impl    │
│                                                                                                  │
│   1515 │   │   if self._compiled_call_impl is not None:                                          │
│   1516 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1517 │   │   else:                                                                             │
│ ❱ 1518 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1519 │                                                                                         │
│   1520 │   def _call_impl(self, *args, **kwargs):                                                │
│   1521 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1527 in _call_impl            │
│                                                                                                  │
│   1524 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1525 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1526 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1527 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1528 │   │                                                                                     │
│   1529 │   │   try:                                                                              │
│   1530 │   │   │   result = None                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/utils/operations.py:553 in forward            │
│                                                                                                  │
│   550 │   model_forward = ConvertOutputsToFp32(model_forward)                                    │
│   551 │                                                                                          │
│   552 │   def forward(*args, **kwargs):                                                          │
│ ❱ 553 │   │   return model_forward(*args, **kwargs)                                              │
│   554 │                                                                                          │
│   555 │   # To act like a decorator so that it can be popped when doing `extract_model_from_pa   │
│   556 │   forward.__wrapped__ = model_forward                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/utils/operations.py:541 in __call__           │
│                                                                                                  │
│   538 │   │   update_wrapper(self, model_forward)                                                │
│   539 │                                                                                          │
│   540 │   def __call__(self, *args, **kwargs):                                                   │
│ ❱ 541 │   │   return convert_to_fp32(self.model_forward(*args, **kwargs))                        │
│   542 │                                                    

これで、GPT-2のような因果言語モデル用のカスタム学習ループが完成し、さらにニーズに合わせてカスタマイズできるようになった。

✏️ 試してみてください！あなたのユースケースに合わせた独自のカスタム損失関数を作成するか、トレーニングループに別のカスタムステップを追加してください。

✏️ 試してみてください！長時間のトレーニング実験を行う際には、TensorBoardやWeights & Biasesなどのツールを使って重要なメトリクスをログに記録することをお勧めします。トレーニングループに適切なロギングを追加することで、トレーニングがどのように進んでいるかを常に確認することができます。


✏️ 試してみてください！長時間のトレーニング実験を行う際には、TensorBoardやWeights & Biasesなどのツールを使って重要なメトリクスをログに記録することをお勧めします。トレーニングループに適切なロギングを追加することで、トレーニングがどのように進んでいるかを常に確認することができます。

